# FOOOF - Model Description

<div class="alert alert-success">
This notebook provides a more theoretical / mathematical description of the model, as well as a detailed description of the parameters.
</div>

### Introduction

In FOOOF, an electrophysiological PSD is fit as a combination of the background process (the arythmic, background 1/f), upon which there may be oscillations - frequency regions in which there are 'bumps', with power over and above this background process.

This formulation roughly translates to fitting the PSD as:

$$psd = BG + \sum_{n=0}^{N} G_n$$

Where $BG$ is the 'background fit' and each $G_n$ is a gaussian fit to a putative oscillation, for $N$ total oscillations extracted from the PSD.

#### Background

The background fit is a log-lorentzian, fit on the semilog PSD (linear frequencies and log10 power values). 

The log-lorentzian is of the form:

$$BG = 10^a * \frac{1}{(b + f^\chi)}$$

Or, equivalently:

$$BG = a - \log(b + f^\chi)$$

In this formulation, the 3 parameters $a$, $b$, and $\chi$ define the background, as:
- $a$ is the 'offset'
- $b$ relates to the 'knee'
- $\chi$ is the 'slope'

#### Oscillations

Regions of power over above this background activity, as defined above, are considered to be putative oscillations, and fit in the model by a gaussian. 

For each gaussian, $n$, with the form:

$$G_n = amp * exp (\frac{- (x - ctr)^2}{2 * wdt^2})$$

The oscillation is defined in terms of the $amp$, $ctr$ and $wdt$, where:
- $amp$ is the amplitude of the oscillation, over and above the background
- $ctr$ is the center frequency of the oscillation
- $wdt$ is the width of the center frequency

The full PSD fit is therefore the combination of the background fit, and $N$ 'oscillatory regions', where each $n$ is a gaussian process found gaussian processes.

Full method details available in the paper: [LINK WHEN AVAILABLE]

## Algorithmic Description

PICTURE!

Briefly, the FOOOF model proceeds as such:
- An intial background fit is taken across the PSD
- This background fit is subtracted from the PSD, creating a flattened spectrum
- Putative oscillations are iteratively found in this flattened spectrum
- A full oscillation fit is created of all oscillation candidates found
- The oscillation fit is subtracted from the original PSD, creating an oscillation-removed PSD
- A final background fit is taken of the oscillation-removed PSD

<div class="alert alert-info">
To step through the algorithm, with the code, go [here](Internals.ipynb).
</div>

FOOOF MODEL FIT PICTURE (plot)

## FOOOF Parameters

There are a number of parameters that control the FOOOF fitting algorithm. Parameters that are exposed to be set by the user are explained in detail below. 

### Background Fitting Approach

#### bg_use_knee (boolean)

Whether to fit a 'knee' parameter in the background fit procedure. 

Default: False

### Controlling Oscillation Fits

#### bandwidth_limits (Hz)

Enforced limits on the bandwidths of extracted oscillations, given as a list of [minimum bandwidth, maximum bandwidth]. We recommend bandwidths at last twice the frequency resolution of the power spectrum.

Default: [0.5, 12]

### Oscillation Search Stopping Criteria

An iterative procedures searches for candidate oscillations in the flattened spectrum. Candidate oscillations are extracted in order of decreasing amplitude, and until some stopping criterion is met. 

#### max_n_oscs (int)

The maximum number of oscillations that can be extracted from a given PSD. FOOOF will halt searching for new oscillations when this number is reached. Note that FOOOF extracts oscillations iteratively by amplitude (over and above the background), and so this approach should extract (up to) the _n_ largest oscillations. 

Default: infinite

#### amp_std_thresh (in units of standard deviation)

The threshold, in terms of standard deviation of the background removed power spectrum, above which a data point must pass to be considered a candidate oscillation. Once a candidate oscillation drops below this threshold, the oscillation search is halted (without including the most recent candidate). 

Default: 2.0

#### min_amp (in units of the psd)

The minimum amplitude, above the background fit, that an oscillation must have to be extracted in the initial fit stage. Once a candidate oscillation drops below this threshold, the oscillation search is halted (without including the most recent candidate). Note that because this constraint is enforced during oscillation search, and prior to final oscillation fit, returned oscillations are not guaranteed to surpass this value in amplitude.

Default: 0

Note: there are two different amplitude-related halting conditions for the oscillation search. By default, the relative (standard-deviation based) threshold is defined, whereas the absolute threshold is set to zero (this is largely because there is no general way to set this value without knowing the scale of the data). If both are defined, both are used and the oscillation search will halt when a candidate oscillations fails to pass either the absolute, or relative threshold. 

<div class="alert alert-info">
To continue with the tutorial, to a hands-on introduction to the codebase, go [here](02-FOOOF.ipynb).
</div>